# Processing Images with OpenCV


When working with images, you will find you need to alter them: be it by applying artistic filters, extrapolating certain sections, blending two images, or whatever else your mind can conjure. This module presents some techniques that you can use to alter
images. 

By the end of it, you should be able to perform tasks such as
- sharpening an image,
- marking the contours of subjects, 
- and detecting crosswalks using a line segment detector. 

Specifically, our discussion and code samples will cover the following topics:

- Converting images between different color models
- Understanding the importance of frequencies and the Fourier transform in image processing
- Applying high-pass filters (HPFs), low-pass filters (LPFs), edge detection filters,and custom convolution filters
- Detecting and analyzing contours, lines, circles, and other geometric shapes
- Writing classes and functions that encapsulate the implementation of a filter

# Converting images between different color models

OpenCV implements literally hundreds of formulas that pertain to the conversion of color
models. Some color models are commonly used by input devices such as cameras, while
other models are commonly used for output devices such as televisions, computer displays,
and printers. In between input and output, when we apply computer vision techniques to
images, we will typically work with three kinds of color models: 

- **Grayscale** is a model that reduces color information by translating it into shades of gray or brightness. This model is extremely useful for the intermediate processing of images in problems where brightness information alone is sufficient, such as face detection. Typically, each pixel in a grayscale image is represented by a single 8-bit value, ranging from 0 for black to 255 for white.
- **BGR** is the blue-green-red color model, in which each pixel has a triplet of values representing the blue, green, and red components or channels of the pixel's color. Web developers, and anyone who works with computer graphics, will be familiar with a similar definition of colors, except with the reverse channel order, red-green-blue (RGB). Typically, each pixel in a BGR image is represented by a triplet of 8-bit values, such as `[0, 0, 0]` for black, `[255, 0, 0]` for blue, `[0,255, 0]` for green, `[0, 0, 255]` for red, and `[255, 255, 255]` for white. 
- The **HSV** model uses a different triplet of channels. Hue is the color's tone, saturation is its intensity, and value represents its brightness.

By default, OpenCV uses the BGR color model (with 8 bits per channel) to represent any image that it loads from a file or captures from a camera.


## Exploring the Fourier transform

OpenCV implements a number of algorithms that enable us to process
images and make sense of the data contained in them, and these are also reimplemented in NumPy to make our life even easier. NumPy has a fast **Fourier transform (FFT)** package, which contains the `fft2` method. This method allows us to compute a discrete **Fourier transform (DFT)** of the image.

Let's examine the concept of the magnitude spectrum of an image using the Fourier
transform. The magnitude spectrum of an image is another image that provides a
representation of the original image in terms of its changes. Think of it as taking an image
and dragging all the brightest pixels to the center. Then, you gradually work your way out
to the border where all the darkest pixels have been pushed. Immediately, you will be able
to see how many light and dark pixels are contained in your image and the percentage of
their distribution.
The Fourier transform is the basis of many algorithms that are used for common image
processing operations, such as edge detection or line and shape detection.
Before examining these in detail, let's take a look at two concepts that – in conjunction with
the Fourier transform – form the foundation of the aforementioned processing operations:
HPFs and LPFs.

## HPFs and LPFs
An HPF is a filter that examines a region of an image and boosts the intensity of certain
pixels based on the difference in the intensity of the surrounding pixels.
Take, for example, the following kernel:
```
[[ 0, -0.25, 0 ],
[-0.25, 1, -0.25],
[ 0, -0.25, 0 ]]
```
A **kernel** is a set of weights that are applied to a region in a source image
to generate a single pixel in the destination image. For example, if we call
an OpenCV function with a parameter to specify a kernel size or `ksize` of
7, this implies that `49 (7 x 7)` source pixels are considered when generating
each destination pixel. We can think of a kernel as a piece of frosted glass
moving over the source image and letting a diffused blend of the source's
light pass through.

The preceding kernel gives us the average difference in intensity between the central pixel
and all its immediate horizontal neighbors. If a pixel stands out from the surrounding
pixels, the resulting value will be high. This type of kernel represents a so-called high-boost
filter, which is a type of HPF, and it is particularly effective in edge detection.

Let's go through an example of applying an HPF to an image:



In [ ]:
import cv2
import numpy as np
from scipy import ndimage

kernel_3x3 = np.array([[-1, -1, -1],
 [-1, 8, -1],
 [-1, -1, -1]])
kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
 [-1, 1, 2, 1, -1],
 [-1, 2, 4, 2, -1],
 [-1, 1, 2, 1, -1],
 [-1, -1, -1, -1, -1]])
img = cv2.imread("../images/statue_small.jpg", 0)
k3 = ndimage.convolve(img, kernel_3x3)
k5 = ndimage.convolve(img, kernel_5x5)
blurred = cv2.GaussianBlur(img, (17,17), 0)
g_hpf = img - blurred
cv2.imshow("3x3", k3)
cv2.imshow("5x5", k5)
cv2.imshow("blurred", blurred)
cv2.imshow("g_hpf", g_hpf)
cv2.waitKey()
cv2.destroyAllWindows()
